In [ ]:
from autoencoder import Autoencoder
from parser import data_converter, print_letter
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
alphabet = np.array(data_converter("resources"))
# for char in alphabet:
#     print_letter(char)
# print(len(alphabet))
flattened_input = np.array(list(map(lambda char: np.array(char).flatten(), alphabet)))

In [ ]:
epochs = []
errors =[]

def callback_fun(e, error, w, o):
    epochs.append(e)
    errors.append(error)

In [ ]:

activation_function = (lambda x: 1/(1+np.exp(-2*x)))
activation_derivative = (lambda x: 2*(1/(1+np.exp(-2*x)))*(1-(1/(1+np.exp(-2*x)))) )
ae = Autoencoder(35, [17, 8], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.01)


output, err_min = ae.train(flattened_input, expected_output=flattened_input, epoch_limit=1500, callback=callback_fun)

print(err_min)

In [ ]:
plt.plot(epochs, errors)

In [ ]:
i = 10
print_letter(alphabet[i])
reconstruct, reconstruction_error = ae.reconstruct(flattened_input[i])

print_letter(reconstruct.reshape(alphabet[i].shape))
print(reconstruction_error)

latent_output = ae.encode(flattened_input[i])
print(latent_output)
print_letter(ae.decode(latent_output).reshape(alphabet[i].shape))

ae.graph_latent_space(flattened_input)

